## Connection to Thymio

In [1]:
# Connect to Thymio with ClientAsync
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()

# Lock Thymio
aw(node.lock())

Node fe36b4a4-2452-4ab5-8d68-b28bb42f53d5

## Importation of libraries

In [2]:
import keyboard
import time
from kalman_filtering.kalman_filter import *
from thymio.control import *
from computer_vision.vision import *
from path_planning.path_plan import *
from navigation.collision_avoidance import *

[[3 3]
 [4 4]
 [5 5]]


## Main script

In [ ]:
map = Map()
if P_VISION:
    print(P_SETUP)
    print("Place the Thymio and the destination, check that everything is properly set. Then, press S.")

# Call a first single update step to create our vision windows
map.update(True) 

while True:
    
    if keyboard.is_pressed('space'):
        map.update(True)
    
    if keyboard.is_pressed('s'):
        if map.found_corners and map.found_destination:
            map.update()
            break
        else:
            print("USER: Must find corners before start")
    
    cv.waitKey(10)

if P_VISION: print(P_START)
if P_VISION: map.info()
await node.wait_for_variables()

vel_old = np.zeros(2)
mu_est_old = np.array([map.robot[0], map.robot[1], map.robot[2]])
cov_est_old = 100*np.eye(3)
view_blocked = False
active = True
end = False
start = True
path = None
segment_index = 0
avoidance_mode = 0
obj_right = False
obstacle_pos = np.array([0, 0])
destination = map.destination[0:2]
pos_robot = map.robot[0:2]
dt = 0.12

while active == True:
    start_time = time.time()
    print("----------------------")
    active = await stop_thymio(node)

    map.update()
    
    # State variables measurement
    await node.wait_for_variables()
 
    # Filtering
    y = map.robot
    x_est, cov_est = kalman_filter(y, vel_old, mu_est_old, cov_est_old, map.found_robot, dt)
    map.pose_est = np.array([int(x_est[0]), int(x_est[1]), x_est[2]], dtype=object)
    cov_predict_old = cov_est
    mu_est_old = map.pose_est

    # Check if final destination moved
    change_dest = ((map.destination[1]-destination[1])**2 + (map.destination[0]-destination[0])**2)*0.5
    if change_dest > 20:
        start = True
        print("Nouveau chemin destination changé")

    #  Check if the robot has been relocated
    change_robot = ((map.pose_est[1]-pos_robot[1])**2 + (map.pose_est[0]-pos_robot[0])**2)*0.5
    if change_robot > 200:
        start = True
        print("Nouveau chemin robot changé")
   
    # Global Navigation
    if start == True:
        destination = map.destination[0:2]
        ex_path, corners = compute_visibility_matrix(map.pose_est[0:2], destination, map.obstacles)
        map.obstacles_lines = possible_lignes(ex_path, corners)
        map.target_lines = a_star_search(corners, ex_path)
        path = path_functions(map.target_lines)
        segment_index = 0
        start = False

    #local nav
    u, avoidance_mode, segment_index, obj_right, obstacle_pos = await collision_avoidance(path, node, map.pose_est, avoidance_mode, segment_index, obj_right, obstacle_pos, map.destination[0:2]) # Zacharie
    #print("avoidance_mode", avoidance_mode)

    if not avoidance_mode:
        u, segment_index, end = path_direction(map.pose_est, path, segment_index)
    
    #update robot position            
    pos_robot = map.pose_est[0:2]

    await node.set_variables(motors(int(u[0]), int(u[1])))

    vel_old = u
    
    end_time = time.time()
    # Update execution time
    dt = end_time - start_time
    cv.waitKey(10)
    if keyboard.is_pressed('q'):
        map.__del__()
        break

----------------------
distance_to_obstacle 643.6342128880348
----------------------
distance_to_obstacle 644.137407701183
----------------------
distance_to_obstacle 642.3122293713549
----------------------
distance_to_obstacle 641.8107197608965
----------------------
distance_to_obstacle 639.9882811427096
----------------------
distance_to_obstacle 640.8158861950911
----------------------
distance_to_obstacle 639.4137940332536
----------------------
distance_to_obstacle 638.5052858042759
----------------------
distance_to_obstacle 638.9240956483015
----------------------
distance_to_obstacle 638.4371543072975
----------------------
distance_to_obstacle 638.8591394039847
----------------------
distance_to_obstacle 638.8591394039847
----------------------
distance_to_obstacle 639.2824102069444
----------------------
distance_to_obstacle 638.3768479511142
----------------------
distance_to_obstacle 638.3768479511142
----------------------
distance_to_obstacle 638.8020037539018
---------

AttributeError: 'NoneType' object has no attribute 'copy'